In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./datasets/MNIST/train.csv')
test = pd.read_csv('./datasets/MNIST/test.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = train.drop(labels = ['label'], axis=1)
Y = train['label']
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Normalize

In [4]:
X = X/255.0
test = test/255.0

In [5]:
X.shape

(42000, 784)

## Train-test split

In [6]:
train = int(0.7*X.shape[0])
test = X.shape[0] - train
print(train, test)

29399 12601


In [7]:
X_train = X[:train]
X_test = X[train:]
print(X_train.shape, X_test.shape)

(29399, 784) (12601, 784)


## Model

In [17]:
# CHECK : Constants
omega = 1.

class ELM(object):
    def __init__(self, sess, batch_size, input_len, hidden_num, output_len):
        '''
        Args:
          sess : TensorFlow session.
          batch_size : The batch size (N)
          input_len : The length of input. (L)
          hidden_num : The number of hidden node. (K)
          output_len : The length of output. (O)
        '''
    
        self._sess = sess 
        self._batch_size = batch_size
        self._input_len = input_len
        self._hidden_num = hidden_num
        self._output_len = output_len 

        # for train
        self._x0 = tf.placeholder(tf.float32, [self._batch_size, self._input_len])
        self._t0 = tf.placeholder(tf.float32, [self._batch_size, self._output_len])

        # for test
        self._x1 = tf.placeholder(tf.float32, [None, self._input_len])
        self._t1 = tf.placeholder(tf.float32, [None, self._output_len])

        self._W = tf.Variable(
          tf.random_normal([self._input_len, self._hidden_num]),
          trainable=False, dtype=tf.float32)
        self._b = tf.Variable(
          tf.random_normal([self._hidden_num]),
          trainable=False, dtype=tf.float32)
        self._beta = tf.Variable(
          tf.zeros([self._hidden_num, self._output_len]),
          trainable=False, dtype=tf.float32)
        self._var_list = [self._W, self._b, self._beta]

        self.H0 = tf.matmul(self._x0, self._W) + self._b # N x L
        self.H0_T = tf.transpose(self.H0)

        self.H1 = tf.matmul(self._x1, self._W) + self._b # N x L
        self.H1_T = tf.transpose(self.H1)

        # beta analytic solution : self._beta_s (K x O)
        if self._input_len < self._hidden_num: # L < K
            identity = tf.constant(np.identity(self._hidden_num), dtype=tf.float32)
            self._beta_s = tf.matmul(tf.matmul(tf.matrix_inverse(
                tf.matmul(self.H0_T, self.H0) + identity/omega), 
                self.H0_T), self._t0)
          # _beta_s = (H_T*H + I/om)^(-1)*H_T*T
        else:
            identity = tf.constant(np.identity(self._batch_size), dtype=tf.float32)
            self._beta_s = tf.matmul(tf.matmul(self.H0_T, tf.matrix_inverse(
                tf.matmul(self.H0, self.H0_T)+identity/omega)), self._t0)
          # _beta_s = H_T*(H*H_T + I/om)^(-1)*T

        self._assign_beta = self._beta.assign(self._beta_s)
        self._fx0 = tf.matmul(self.H0, self._beta)
        self._fx1 = tf.matmul(self.H1, self._beta)

        self._cost = tf.reduce_mean(tf.cast(tf.losses.mean_squared_error(labels=self._t0, predictions=self._fx0), tf.float32))
                                        
        self._init = False
        self._feed = False

        # for the mnist test
        self._correct_prediction = tf.equal(tf.argmax(self._fx1,1), tf.argmax(self._t1,1))
        self._accuracy = tf.reduce_mean(tf.cast(self._correct_prediction, tf.float32))
        self._cost = tf.reduce_mean(tf.cast(tf.losses.mean_squared_error(labels=self._t1, predictions=self._fx1), tf.float32))


    def feed(self, x, t):
        '''
        Args :
          x : input array (N x L)
          t : label array (N x O)
        '''

        if not self._init : self.init()
        self._sess.run(self._assign_beta, {self._x0:x, self._t0:t})
        print(self._sess.run(self._cost, {self._x0:x, self._t0:t}))
        self._feed = True

    def init(self):
        self._sess.run(tf.initialize_variables(self._var_list))
        self._init = True

    def test(self, x, t=None):
        if not self._feed : exit("Not feed-forward trained")
        if t is not None :
            print("Accuracy: {:.9f}".format(self._sess.run(self._accuracy, {self._x1:x, self._t1:t})))
            print(self._sess.run(self._cost, {self._x1:x, self._t1:t}))
        else :
            return self._sess.run(self._fx1, {self._x1:x})


In [18]:
# Basic tf setting
tf.set_random_seed(2016)
sess = tf.Session()

In [19]:
## training for outlier detetcion
batch_size = 5000
hidden_num = 150
print("batch_size : {}".format(batch_size))
print("hidden_num : {}".format(hidden_num))
elm = ELM(sess, batch_size, 784, hidden_num, 784)

# one-step feed-forward training
train_x, train_y = (X_train[:batch_size], X_train[:batch_size])
print(X_train[:batch_size].shape)
elm.feed(train_x, train_y)

batch_size : 5000
hidden_num : 150
(5000, 784)


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_11' with dtype float and shape [?,784]
	 [[{{node Placeholder_11}} = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_11', defined at:
  File "c:\users\asus\appdata\local\programs\python\python36\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\asus\appdata\local\programs\python\python36\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\asus\envs\tf\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\asus\envs\tf\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\asus\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "c:\users\asus\envs\tf\lib\site-packages\tornado\platform\asyncio.py"

In [ ]:
elm.test(X_test, X_test)